 k-최근접(k-nn) 이웃 분류모델은 훈련단계에서 학습을 하지 않음
테스트/검증 단계에서 테스트 관측값과 가장 근접한 훈련 관측값을 비교
거리에만 의존하니까 차원의저주! 이걸 보완하려면 예측에 필요한 특징의 개수가 늘어나야하고 -> 성능이 저하됨


In [7]:
import pandas as pd
breast_cancer = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data",  header =None)
breast_cancer.head()

,0,1,2,3,4,5,6,7,8,9,10
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [15]:
# 컬럼명주기
breast_cancer.columns = ['id_number', 'cluster_thickness','unif_cell_size', 'unif_cell_shape', 'marg_adhesion', 'single_epith_cell_size',  'bare_nuclei', 'bland_chromatin', 'normal_nucleoli', 'mitoses', 'class']
breast_cancer.head()

,id_number,cluster_thickness,unif_cell_size,unif_cell_shape,marg_adhesion,single_epith_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [16]:
## bare_nuclei       의 데이터 값이 문자일수도
breast_cancer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id_number               699 non-null    int64 
 1   cluster_thickness       699 non-null    int64 
 2   unif_cell_size          699 non-null    int64 
 3   unif_cell_shape         699 non-null    int64 
 4   marg_adhesion           699 non-null    int64 
 5   single_epith_cell_size  699 non-null    int64 
 6   bare_nuclei             699 non-null    object
 7   bland_chromatin         699 non-null    int64 
 8   normal_nucleoli         699 non-null    int64 
 9   mitoses                 699 non-null    int64 
 10  class                   699 non-null    int64 
dtypes: int64(10), object(1)
memory usage: 60.2+ KB


In [17]:
## 데이터 프레임 전체에서 null의 개수
breast_cancer.isnull().values.sum()

0

In [24]:
import numpy as np

# ?을 NA로 바꾼후, NA를 최빈값으로 만들기
breast_cancer["bare_nuclei"] = breast_cancer["bare_nuclei"].replace('?', np.NAN)

breast_cancer["bare_nuclei"]= \
    breast_cancer["bare_nuclei"].fillna(breast_cancer["bare_nuclei"].value_counts().index[0])

In [27]:
# class를 0과 1로 만들기
breast_cancer["cancer_ind"] = 0
breast_cancer.loc[breast_cancer['class']==4, 'cancer_ind']= 1
# X,y만들기
X = breast_cancer.drop(["id_number", "class", "cancer_ind"], axis = 1)
y = breast_cancer.cancer_ind

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=42)


In [29]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [30]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [32]:
from sklearn.neighbors import KNeighborsClassifier


In [33]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier(n_neighbors=3)

In [34]:
knn.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 3,
 'p': 2,
 'weights': 'uniform'}

In [36]:
## 분류모델의 정확도 AUC
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve

y_pred = knn.predict(X_test_scaled)
accuracy_score(y_test, y_pred)

0.9761904761904762

In [37]:
## 분류모델의 성능평가(모델평가) : confusion(https://bcho.tistory.com/1206 가서공부)
confusion_matrix(y_test, y_pred)

array([[141,   2],
       [  3,  64]], dtype=int64)

In [38]:
roc_auc_score(y_test, y_pred)

0.9706189333055005

In [42]:
### 최적의 grid 찾기
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(knn,{"n_neighbors": [1, 2, 3, 4, 5]}, \
                          n_jobs= -1, cv=7, scoring="roc_auc")
grid_search.fit(X_train_scaled, y_train)

GridSearchCV(cv=7, estimator=KNeighborsClassifier(n_neighbors=3), n_jobs=-1,
             param_grid={'n_neighbors': [1, 2, 3, 4, 5]}, scoring='roc_auc')